# WebCam (USB) Camera Test

In this notebook, you can test your camera to make sure it's working on the Jetson Nano as expected.  It should already be plugged into the USB camera port.  Make sure there is no obstruction on the camera lens such as a film or cover.

### Check to see if the device is available
Execute the following system command to list all video devices on the Jetson Nano.  If your camera doesn't show up with a device id, check your connection.  You should get an output similar to 
```text
crw-rw----+ 1 root video 81, 0 Jun  2 17:35 /dev/video0
```

In [ ]:
!ls -ltrh /dev/video*

### Create the camera object

First, create a camera object by importing the `USBCamera` class from the library by executing the following Python code cell.  Please note, you can only create one `USBCamera` instance. Set the `capture_device=` to the correct number found when you listed the system video devices.  If you have `/dev/video0`, then set `capture_device=0`.  If you have `/dev/video1`, set `capture_device=1` in the code line below.

In [ ]:
from jetcam.usb_camera import USBCamera

#TODO change capture_device if incorrect for your system
camera = USBCamera(width=224, height=224, capture_width=640, capture_height=480, capture_device=0)

We can then capture a frame from the camera with the `read` method. 

In [ ]:
image = camera.read()

print(image.shape)

Calling the `read` method for `camera` also updates the camera's internal `value`.  By looking at the value's `shape`, we see three numbers representing the pixel height, pixel width, and number of color channels.

In [ ]:
print(camera.value.shape)

### Create a widget to view the image stream
We can create a "widget" to display this image in the notebook.  In order to see the image, convert it from its blue-green-red format (brg8) to a format the browser can display (jpeg). 

In [ ]:
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

image_widget = ipywidgets.Image(format='jpeg')

image_widget.value = bgr8_to_jpeg(image)

display(image_widget)

You should see an image from the camera if all is working correctly.  If there seems to be an image but it's fuzzy or a funny color, check to make sure there is no protective film or cap on the lens.  

Now let's watch a live stream from the camera.  Set the `running` value of the camera to continuously update the value in background.  This allows you to attach "callbacks" to the camera value changes.

The "callback" here is the function, `update_image`, which is attached by calling the `observe` method below.  `update_image` is executed whenever there is a new image available to process, which is then displayed in the widget.

In [ ]:
camera.running = True

def update_image(change):
    image = change['new']
    image_widget.value = bgr8_to_jpeg(image)
    
camera.observe(update_image, names='value')

If you move something in front of the camera, you should now see the live video stream in the widget. To stop it, unattach the callback with the `unobserve` method.

In [ ]:
camera.unobserve(update_image, names='value')

### Another way to view the image stream
You can also use the traitlets `dlink` method to connect the camera to the widget, using a transform as one of the parameters.  This eliminates some steps in the process.

In [ ]:
import traitlets

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

You can remove the camera/widget link with the `unlink` method.

In [ ]:
camera_link.unlink()

... and reconnect it again with `link`.

In [ ]:
camera_link.link()

<h1 style="background-color:#76b900;"></h1>

## Before you go...<br><br>Shut down the camera and/or notebook kernel to release the camera resource

In [ ]:
# Attention!  Execute this cell before moving to another notebook
# The USB camera application only requires that the notebook be reset
# The CSI camera application requires that the 'camera' object be specifically released

import os
os._exit(00)